In [1]:
# Import libraries
import pandas as pd
import numpy as np
from scipy import sparse
from sklearn.metrics.pairwise import cosine_similarity
# Ignore printing warnings for general readability
import warnings 
warnings.filterwarnings("ignore")

In [2]:
# Loading the dataset 
def loaddata(filename):
    df = pd.read_csv(f'{filename}.csv',sep=';',error_bad_lines=False,warn_bad_lines=False,encoding='latin-1')
    return df

book   = loaddata("../../BX-Books")
user   = loaddata("../../BX-Users")
rating = loaddata("../../BX-Book-Ratings")

In [3]:
#Preprocessing Data
book = book[['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher']]
book.rename(columns = {'Book-Title':'title', 'Book-Author':'author', 'Year-Of-Publication':'year', 'Publisher':'publisher'}, inplace=True)
user.rename(columns = {'User-ID':'user_id', 'Location':'location', 'Age':'age'}, inplace=True)
rating.rename(columns = {'User-ID':'user_id', 'Book-Rating':'rating'}, inplace=True)

In [4]:
rating

,user_id,ISBN,rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6
...,...,...,...
1149775,276704,1563526298,9
1149776,276706,0679447156,0
1149777,276709,0515107662,10
1149778,276721,0590442449,10


In [5]:
# Let's visualize ratings given by users

rating_users = rating['user_id'].value_counts().reset_index().\
               rename({'index':'user_id','user_id':'rating'}, axis=1)
rating_users

,user_id,rating
0,11676,13602
1,198711,7550
2,153662,6109
3,98391,5891
4,35859,5850
...,...,...
105278,116180,1
105279,116166,1
105280,116154,1
105281,116137,1


In [6]:
# Let's visualize ratings received by books

rating_books = rating['ISBN'].value_counts().reset_index().\
               rename({'index':'ISBN','ISBN':'rating'}, axis=1)
rating_books

,ISBN,rating
0,0971880107,2502
1,0316666343,1295
2,0385504209,883
3,0060928336,732
4,0312195516,723
...,...,...
340551,1568656386,1
340552,1568656408,1
340553,1569551553,1
340554,1570081808,1


In [7]:
# In order to avoid rating bias & for making good recommendations, limit the dataset to only those
# users that have made at least 250 ratings & books that have received at least 50 ratings

rating = rating[rating['user_id'].isin(rating_users[rating_users['rating']>200]['user_id'])]
rating = rating[rating['ISBN'].isin(rating_books[rating_books['rating']> 50]['ISBN'])]

rating


,user_id,ISBN,rating
1456,277427,002542730X,10
1468,277427,006092988X,0
1469,277427,0060930535,0
1470,277427,0060932139,0
1471,277427,0060934417,0
...,...,...,...
1147440,275970,1400031354,0
1147441,275970,1400031362,0
1147470,275970,1558744606,0
1147517,275970,1573229725,0


In [8]:
# For the recommendation system, it is prefered to have the book titles rather than ISBN for easier interpretation

rating = rating.merge(book, on="ISBN")[['user_id','title','rating','ISBN']] # merging with the book dataframe
rating                                                                         # on 'ISBN' to get 'Book-Title'

,user_id,title,rating,ISBN
0,277427,Politically Correct Bedtime Stories: Modern Ta...,10,002542730X
1,3363,Politically Correct Bedtime Stories: Modern Ta...,0,002542730X
2,11676,Politically Correct Bedtime Stories: Modern Ta...,6,002542730X
3,12538,Politically Correct Bedtime Stories: Modern Ta...,10,002542730X
4,13552,Politically Correct Bedtime Stories: Modern Ta...,0,002542730X
...,...,...,...,...
89924,234828,Ringworld,8,0345333926
89925,236283,Ringworld,0,0345333926
89926,249628,Ringworld,0,0345333926
89927,261829,Ringworld,0,0345333926


In [9]:
# Check for duplicate values
print(f'Duplicate entries: {rating.duplicated().sum()}')

Duplicate entries: 0


In [10]:
rating.drop_duplicates(inplace=True)
rating

,user_id,title,rating,ISBN
0,277427,Politically Correct Bedtime Stories: Modern Ta...,10,002542730X
1,3363,Politically Correct Bedtime Stories: Modern Ta...,0,002542730X
2,11676,Politically Correct Bedtime Stories: Modern Ta...,6,002542730X
3,12538,Politically Correct Bedtime Stories: Modern Ta...,10,002542730X
4,13552,Politically Correct Bedtime Stories: Modern Ta...,0,002542730X
...,...,...,...,...
89924,234828,Ringworld,8,0345333926
89925,236283,Ringworld,0,0345333926
89926,249628,Ringworld,0,0345333926
89927,261829,Ringworld,0,0345333926


## User-Based Collaborative Filtering Recommendation Algorithm    

In [11]:
#Create user-item matrix
book_pivot = rating.pivot_table(columns='ISBN', index='user_id', values="rating")
book_pivot

ISBN,000649840X,002026478X,0020442203,002542730X,0028604199,006000438X,0060008032,0060008776,006001203X,0060085444,...,1860492592,1878424319,1885171080,1931561648,3257228007,3257229534,3404148665,3423202327,3442541751,3492045170
user_id,,,,,,,,,,,,,,,,,,,,,
254,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2276,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
2766,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2977,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3363,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
275970,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
277427,NaN,NaN,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
277478,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


    We normalize the rating by extracting the average rating of users. After normalization, the item with a rating less then user's average rating get a negative value (user don't like item), and the item with a rating more than the user's average rating get a positive value (user likes item).

In [12]:
# Normalize user-item matrix
# axis = 1 cloumns
# axis = 0 index 
matrix_norm = book_pivot.subtract(book_pivot.mean(1), axis=0 )
matrix_norm

ISBN,000649840X,002026478X,0020442203,002542730X,0028604199,006000438X,0060008032,0060008776,006001203X,0060085444,...,1860492592,1878424319,1885171080,1931561648,3257228007,3257229534,3404148665,3423202327,3442541751,3492045170
user_id,,,,,,,,,,,,,,,,,,,,,
254,NaN,NaN,NaN,NaN,NaN,-1.909091,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2276,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,-4.035714,NaN,NaN,NaN,NaN
2766,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2977,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3363,NaN,NaN,NaN,-0.890411,-0.890411,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
275970,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
277427,NaN,NaN,NaN,8.054795,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
277478,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
# Matrix with NaN values
matrix_norm_nan = matrix_norm.copy(deep=True)
matrix_norm_nan

ISBN,000649840X,002026478X,0020442203,002542730X,0028604199,006000438X,0060008032,0060008776,006001203X,0060085444,...,1860492592,1878424319,1885171080,1931561648,3257228007,3257229534,3404148665,3423202327,3442541751,3492045170
user_id,,,,,,,,,,,,,,,,,,,,,
254,NaN,NaN,NaN,NaN,NaN,-1.909091,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2276,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,-4.035714,NaN,NaN,NaN,NaN
2766,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2977,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3363,NaN,NaN,NaN,-0.890411,-0.890411,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
275970,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
277427,NaN,NaN,NaN,8.054795,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
277478,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Identify Similar Users

In [14]:
matrix_norm.fillna(0, inplace=True)
matrix_norm

ISBN,000649840X,002026478X,0020442203,002542730X,0028604199,006000438X,0060008032,0060008776,006001203X,0060085444,...,1860492592,1878424319,1885171080,1931561648,3257228007,3257229534,3404148665,3423202327,3442541751,3492045170
user_id,,,,,,,,,,,,,,,,,,,,,
254,0.0,0.0,0.00000,0.000000,0.000000,-1.909091,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
2276,0.0,0.0,0.00000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,-4.035714,0.0,0.0,0.0,0.0
2766,0.0,0.0,0.00000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
2977,0.0,0.0,0.00000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
3363,0.0,0.0,0.00000,-0.890411,-0.890411,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
275970,0.0,0.0,0.00000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
277427,0.0,0.0,0.00000,8.054795,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
277478,0.0,0.0,0.00000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0


In [15]:
# user_similarity = matrix_norm.T.corr()
# user_similarity

In [16]:
user_similarity_matrix = pd.DataFrame(cosine_similarity(matrix_norm), index=matrix_norm.T.columns, columns=matrix_norm.T.columns)
user_similarity_matrix

user_id,254,2276,2766,2977,3363,3757,4017,4385,6242,6251,...,274004,274061,274301,274308,274808,275970,277427,277478,277639,278418
user_id,,,,,,,,,,,,,,,,,,,,,
254,1.000000,-0.016016,-0.002798,0.033965,0.004848,0.0,-0.082784,0.000000,-0.016394,0.063623,...,0.002531,-0.045615,0.009170,0.032551,-0.028994,-0.027925,-0.034256,0.000000,0.000000,-0.001477
2276,-0.016016,1.000000,-0.018555,0.012576,-0.006342,0.0,-0.011696,0.089512,0.000000,0.000203,...,0.005960,-0.017207,-0.025529,-0.006704,0.000000,0.001925,0.010673,0.006842,-0.006306,0.003567
2766,-0.002798,-0.018555,1.000000,-0.019180,0.012346,0.0,0.045174,0.000000,-0.018767,-0.041767,...,-0.002803,0.004021,-0.029962,-0.021462,0.006168,0.033103,-0.047899,0.098643,-0.052885,-0.009247
2977,0.033965,0.012576,-0.019180,1.000000,-0.009183,0.0,0.020483,0.000000,0.030649,0.035695,...,-0.021983,-0.023345,0.048257,0.009690,-0.032235,-0.049652,-0.005767,0.011726,-0.023445,0.002900
3363,0.004848,-0.006342,0.012346,-0.009183,1.000000,0.0,0.034151,0.000000,0.004847,-0.019875,...,0.003188,0.008531,0.002046,-0.008537,0.086984,-0.011058,-0.011186,0.007560,-0.007326,-0.011965
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
275970,-0.027925,0.001925,0.033103,-0.049652,-0.011058,0.0,0.001582,-0.015220,0.016535,0.009892,...,-0.045960,-0.008647,-0.027877,0.001048,-0.043595,1.000000,-0.050134,0.000000,-0.005014,-0.006475
277427,-0.034256,0.010673,-0.047899,-0.005767,-0.011186,0.0,0.022116,0.000000,0.016516,0.011634,...,-0.006760,0.003114,0.019559,0.069339,-0.009832,-0.050134,1.000000,0.004976,0.014397,0.000618
277478,0.000000,0.006842,0.098643,0.011726,0.007560,0.0,0.004288,0.028239,0.000000,0.007142,...,0.004716,-0.039116,0.009791,0.008749,0.000000,0.000000,0.004976,1.000000,-0.006603,0.002177


In the user similarity matrix, the values range from -1 to 1, where -1 means opposite book preference and 1 means same book preference

In [17]:
# number of similar users
k = 40
# Pick a user ID
picked_user = 254
# Get top k similar users
similar_user = user_similarity_matrix[picked_user].sort_values(ascending=False)[1:k+1]
similar_user

user_id
208141    0.305870
175003    0.280318
205735    0.261878
79186     0.248908
179978    0.232578
102702    0.230382
170634    0.208253
78553     0.174384
201783    0.172766
39616     0.172038
28523     0.166064
73681     0.162554
101851    0.146267
88733     0.145590
154176    0.143603
12824     0.136956
167800    0.135221
168047    0.127554
30735     0.126950
164533    0.124479
201290    0.122267
226965    0.120049
141819    0.115394
87555     0.114584
85993     0.109765
170513    0.109331
23872     0.108436
55492     0.107984
8681      0.106899
75860     0.106150
130554    0.106032
240144    0.104210
263163    0.104104
15408     0.102067
206534    0.098975
162639    0.098233
76223     0.096203
81492     0.095771
147141    0.093084
177374    0.090434
Name: 254, dtype: float64

We narrow down items by doing the following:
<ul><li> Remove the books that have been read by target user
    <li> Keep only the movies that similar users have read

In [18]:
# Books that similar users read. Remove that none of the similar users have read
similar_user_book = matrix_norm_nan[matrix_norm_nan.index.isin(similar_user.index)].dropna(axis=1, how='all')
similar_user_book

ISBN,002026478X,0020442203,002542730X,0028604199,0060008032,0060008776,0060085444,0060096195,0060168013,006016848X,...,1573225517,1573225789,1573227331,1573228214,157322930X,1573229326,1573229385,1573229725,1576737330,1885171080
user_id,,,,,,,,,,,,,,,,,,,,,
8681,NaN,NaN,NaN,NaN,NaN,-2.873016,NaN,NaN,NaN,NaN,...,7.126984,NaN,5.126984,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12824,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15408,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23872,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28523,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30735,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-6.295775,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39616,NaN,NaN,NaN,-0.641304,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
55492,-0.642857,9.357143,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.642857,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
73681,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
# Books that the target user has watched
picked_userid_read = matrix_norm_nan[matrix_norm_nan.index == picked_user].dropna(axis=1, how='all')
picked_userid_read

ISBN,006000438X,0060930535,0060934719,0061020710,0064471047,014028009X,0140298479,014100018X,0142001740,0312305060,...,0671525743,0671527215,0679735909,067976402X,0684833395,0684872153,0684874350,0743225406,0804106304,080410753X
user_id,,,,,,,,,,,,,,,,,,,,,
254,-1.909091,-1.909091,-1.909091,-1.909091,5.090909,-1.909091,-1.909091,-1.909091,7.090909,-1.909091,...,-1.909091,-1.909091,-1.909091,-1.909091,-1.909091,-1.909091,-1.909091,-1.909091,-1.909091,-1.909091


In [20]:
# Remove the read books from the book list
similar_user_book.drop(picked_userid_read.columns,axis=1, inplace = True,errors = 'ignore')
similar_user_book

ISBN,002026478X,0020442203,002542730X,0028604199,0060008032,0060008776,0060085444,0060096195,0060168013,006016848X,...,1573225517,1573225789,1573227331,1573228214,157322930X,1573229326,1573229385,1573229725,1576737330,1885171080
user_id,,,,,,,,,,,,,,,,,,,,,
8681,NaN,NaN,NaN,NaN,NaN,-2.873016,NaN,NaN,NaN,NaN,...,7.126984,NaN,5.126984,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12824,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15408,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23872,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28523,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30735,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-6.295775,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39616,NaN,NaN,NaN,-0.641304,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
55492,-0.642857,9.357143,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.642857,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
73681,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
item_score = {}

for i in similar_user_book.columns:
    book_rating = similar_user_book[i]
    # create a variable to score the score
    total = 0
    # variable to store the number of score
    divide = 0
    for u in similar_user.index:
        if pd.isna(book_rating[u]) == False:
            score = similar_user[u] * book_rating[u]
            total +=score
#             divide += abs(book_rating[u])
            divide+=1
    # get average score for item 
    item_score[i] = total/(divide+1e-8)

m = 10
item_score = pd.DataFrame(item_score.items(), columns=['ISBN', 'book_score'])
ranked_item_score = item_score.sort_values(by= 'book_score', ascending=False)

ranked_book = ranked_item_score.head(10)
ranked_book

,ISBN,book_score
277,0345392825,1.863076
716,0440236991,1.819429
1372,0684814994,1.741749
1298,0671727583,1.601198
1277,0671534718,1.599135
772,0446525537,1.358664
1303,0671744216,1.265977
1544,155874262X,1.208960
229,0345285549,1.173130
1451,0786889020,1.164925


In [22]:
book_recommend = pd.merge(book, ranked_book, on="ISBN").sort_values(by= 'book_score', ascending=False)
book_recommend

,ISBN,title,author,year,publisher,book_score
2,0345392825,Reviving Ophelia: Saving the Selves of Adolesc...,Mary Pipher,1995,Ballantine Books,1.863076
8,0440236991,Leap of Faith,Danielle Steel,2002,Dell Publishing Company,1.819429
9,0684814994,Christmas Box (Christmas Box Trilogy),Richard Paul Evans,1995,Simon &amp; Schuster,1.741749
4,0671727583,Smart Women,Judy Blume,1990,Pocket Books,1.601198
6,0671534718,Melody (Logan),V.C. Andrews,1996,Pocket,1.599135
5,0446525537,A Walk to Remember,Nicholas Sparks,1999,Warner Books,1.358664
7,0671744216,The Secret,Julie Garwood,1992,Pocket,1.265977
0,155874262X,Chicken Soup for the Soul (Chicken Soup for th...,Jack Canfield,1993,Health Communications,1.208960
1,0345285549,Elfstones of Shannara,Terry Brooks,1990,Del Rey Books,1.173130
3,0786889020,Don't Stand Too Close to a Naked Man,Tim Allen,1995,Hyperion,1.164925
